https://github.com/ratsgo/ratsnlp/tree/master/ratsnlp/nlpbook

## 트랜스퍼 러닝
- 트랜스퍼 러닝(transfer learning)이란 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법
- 트랜스퍼 러닝이 주목받게 된 것은 업스트림 태스크와 프리트레인 덕분. 자연어의 풍부한 문맥(context)을 모델에 내재화하고 이 모델을 다양한 다운스트림 태스크에 적용해 그 성능을 대폭 끌어올림
- 대표적인 업스트림 태스크 가운데
1. 다음 단어 맞추기.
- GPT 계열 모델이 바로 이 태스크로 프리트레인을 수행
- 문맥이 주어졌을 때, 학습 데이터 말뭉치에 해당 구가 많다고 하면 모델은 이를 바탕으로 다음에 올 단어를 분류하도록 학습
- 대규모 말뭉치를 가지고 반복 수행하면 이전 문맥을 고려했을 때 어떤 단어가 그 다음에 오는 것이 자연스러운지 알 수 있게 됨
  * 해당하는 단어의 확률은 높이고, 나머지 단어들의 확률을 낮추는 방향으로 모델 전체를 업데이트하는 것


2. 빈칸 채우기.
##### 업스트림 태스트
- BERT 계열 모델이 바로 이 태스크로 프리트레인을 수행.
- 문장에서 빈칸을 만들고 해당 위치에 들어갈 단어가 무엇ㅇ리지 맞히는 과정에서 학습
  * 많은 양의 데이터를 가지고 빈칸 채우기를 반복 학습하면 앞뒤 문맥을 보고 빈칸에 적합한 단어를 알 수 있는 것.
- ‘빈칸 채우기’로 업스트림 태스크를 수행한 모델을 **마스크 언어 모델**(Masked Language Model)
  * 해당하는 단어의 확률은 높이고 나머지 단어아 관계된 확률은 낮추는 방향으로 모델 전체를 업데이트함
- 다음 단어 맞히기, 빈칸 채우기 같은 업스트림 태스크는 강력한 힘을 지니며 뉴스, 웹 문서, 백과사전 등 글만 있으면 수작업 없이도 다량의 학습 데이터를 아주 싼값에 만들어 낼 수 있다.
- 이처럼 데이터 내에서 정답을 만들고 이를 바탕으로 모델을 학습하는 방법을 자기 지도 학습(self-supervised learning)이라고 함.


##### 다운스트림 태스크
- 다운스트림 태스크의 학습 방식은 모두 **파인튜닝**(fine-tuning)이며, 프리트레인을 마친 모델을 다운스트림 태스크에 맞게 업데이트하는 기법
1. 문서 분류
- 자연어(문서나 문장)를 입력받아 해당 입력이 어떤 범주에 속하는지 그 확률값을 반환
2. 자연어 추론
- 문장 2개를 입력받아 두 문장 사이의 관계가 참, 거짓, 중립 등 어떤 범주인지 그 확률값을 반환
3. 개체명 인식
- 자연어(문서나 문장)를 입력받아 단어별로 어떤 개체명 범주에 속하는지 그 확률값을 반환
4. 질의응답
- 자연어(질문+지문)을 입력받아 각 단어가 정답의 시작인 확률과 끝일 확률값을 반환
5. 문장생성
- GPT 계열 언어 모델이 널리 쓰임
- 자연어(문장)를 입력받아 어휘 전체에 대한 확률값을 반환.
  * 이 확률값은 입력된 문장 다음에 올 단어로 얼마나 적절한 지 나타내는 점수

# 2장 문장을 작은 단위로 쪼개기

## 2-1 토큰화란?
- 토큰화: 토큰 시퀀스로 나누는 과정

1) 단어
- 어절 단위로 토큰화 수행. 즉, 공백으로 분리
  * 어휘 집합의 크기가 매우 커질 수 있기에 의미 있는 단위로 토큰화하면 됨
2) 문자
- 각 문자 토큰은 의미 있는 단위가 되기 어려움 (어제-> 어, 제 이렇게 분리하니까)
- 단어 단위와 비교할 때, 분석 결과인 토큰 시퀀스의 길이가 상대적으로 길다. 그러면 모델이 해당 문장을 학습하기가 어려워지고 결과적으로 성능이 떨어지게 됨

3) 서브워드
- 단어와 문자 단위 토큰화의 중간

## 2-2 바이트 페어 인코딩이란? _ GPT 모델
- 원래 정보를 압축하는 알고리즘으로 제안되었는데 최근에는 자연어 처리 모델에 널리 쓰이고 있는 토큰화 기법
- 데이터에서 가장 많이 등장한 문자열을 병합(연속된 두 글자를 한 글자로 병합)해서 데이터를 압축하는 기법


**BPE 어휘 집합 구축하기**

- 말뭉치 준비

1. 말뭉치의 모든 문장을 공백으로 나눠줌 "프리토크나이즈"
2. 나뉜 토큰을 2개씩 묶어서 나열 "바이그램"
  * 바이그램 쌍이 같은 것끼리 그 빈도를 합쳐주는 것
3. 가장 많이 등장한 바이그램 쌍을 어휘 집합에 추가
4. BPE 어휘 집합은 사용자가 정한 크기가 될 때까지 반복,수행
  * 이 어휘집합은 허깅페이스 tokenizers 패키지를 활용한 실습해서 vocab.json 파일로 저장
  * 이를 merge.txt라는 자료로 만들어 BPE 토큰화 과정에서 서브워드 병합 우선순위를 정함

**BPE 토큰화**
- 어휘집합(vocab.json)과 병합 우선순위(merge.txt)가 있으면 토큰화를 수행할 수 있음
1. 문자 단위로 분리
2. merges.txt 파일을 참고해 병합 우선순위 부여
3. 우선순위가 높은 문자를 먼저 합쳐줌
4. 다시 파일을 참고해 병합 우선순위를 부여

즉,

step1 어절별로 병합 우선순위가 높은 바이그램 쌍을 반복해서 병합

step2 병합된 토큰이 어휘 집합에 있는지 확인해 최종 결과 도출

**워드피스**_ BERT 모델
- 말뭉치에서 자주 등장한 문자열을 토큰으로 인식한다는 점에서 BPE와 본질적으로 유사.
- 다만 어휘 집합을 구축할 때 문자열을 병합하는 기준이 다름.
  * 워드피스는 BPE처럼 단순히 빈도를 기준으로 병합하는 것이 아니라, 병합했을 때 말뭉치의 우도(likelihood)를 가장 높이는 쌍을 병합
  * 워드피스는 어휘 집합만을 가지고 토큰화 함

    - 분석 대상 어절에 어휘 집합에 잇는 서브 워드가 포함돼 있을 때, 해당 서브워드를 어절에서 분리
    - 단, 서브워드 후보가 여럿 있을 때, 가장 긴 서브워드를 선택
    - 이후 어절의 나머지에서 어휘 집합에 잇는 서브워드를 다시 찾고 또 분리
    - 분석 대상 문자열에서 서브워드 후보가 하나도 없으면 해당 문자열 전체를 미등록 단어로 취급

## 2-3 어휘 집합 구축하기

**BPE 기반 토크나이저 만들기**

1단계 실습 환경 만들기

In [6]:
!pip install ratsnlp

2단계 구글 드라이브 연동

- 코랩 노트북은 일정 시간 사용하지 않으면 모든 결과물이 날아갈 수 있기에 구축할 어휘 집합을 따로 저장하기 위함

In [7]:
# 구글 드라이브와 연동

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


3단계 말뭉치 내려받기 및 전처리
- 오픈소스 파이썬 패키지 코포라를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리함

In [8]:
# 말뭉치 내려받기

# NSMC 다운로드
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 111MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 56.7MB/s]


In [9]:
# NSMC 전처리
import os

def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

# 데이터셋에서 모든 텍스트를 가져옵니다.
train_texts = nsmc.train.get_all_texts()
test_texts = nsmc.test.get_all_texts()

# 텍스트를 파일로 저장합니다.
write_lines("/content/train.txt", train_texts)
write_lines("/content/test.txt", test_texts)


4단계 GPT 토크나이저 구축
- GPT 계열 모델이 사용하는 토크나이저 기법은 BPE.
단, 유니코드 바이트 수준으로 어휘 집합을 구축하고 토큰화를 수행
- 바이트 수준으로 BPe를 수행한다는 것은 어휘 ㅈ비합 구축 대상 말뭉치를 위와 같이 변환하고 이들을 문자 취급해 가장 자주 등장하는 문자열을 병합하는 방식으로 어휘 집합을 만든다는 것.

In [10]:
# 디렉터리 만들기, BPE 어휘 집합 구축 결과를 저장할 디렉터리

import os
os.makedirs("gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

In [2]:
# 바이트 레벨 BPE 어휘 집합 구축
!pip install tokenizers

from tokenizers import ByteLevelBPETokenizer

# Byte-Level BPE 토크나이저 객체 생성
bytebpe_tokenizer = ByteLevelBPETokenizer()

# 학습 데이터로 토크나이저 학습
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,          # 어휘 집합 크기 조절
    special_tokens=["[PAD]"]   # 특수 토큰 추가
)

# 학습된 토크나이저 모델 저장
bytebpe_tokenizer.save_model("/content/gdrive/My Drive/nlpbook/bbpe")


['/content/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/content/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [3]:
# 바이트 레벨 BPE 어휘 집합 구축 결과 (VOCAB.JSON)

!cat /gdrive/MyDrive/nlpbook/bbpe/vocab.json

cat: /gdrive/MyDrive/nlpbook/bbpe/vocab.json: No such file or directory


In [4]:
# 바이트 레벨 BPE 병합 우선순위 (MERGE.TXT)

!cat /gdrive/MyDrive/nlpbook/bbpe/merge.TXT

cat: /gdrive/MyDrive/nlpbook/bbpe/merge.TXT: No such file or directory


5단계 BERT 토크나이저 구축

In [5]:
# 디렉터리 만들기
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

In [7]:
# 워드피스 어휘 집합 구축

from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size = 10000,
)

wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

In [8]:
# 워드피스 어휘 집합 구축 결과

! cat /gdrive/MyDrive/nlpbook/wordpiece/vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
!
"
%
&
'
(
)
*
+
,
-
.
/
0
1
2
3
4
5
6
7
8
9
:
;
<
=
>
?
@
A
B
C
D
E
F
G
I
K
L
M
N
O
P
R
S
T
V
W
X
[
]
^
_
`
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
r
s
t
u
v
w
x
y
z
~
★
♡
♥
ㄱ
ㄴ
ㄷ
ㄹ
ㅁ
ㅂ
ㅅ
ㅇ
ㅈ
ㅉ
ㅋ
ㅎ
ㅏ
ㅗ
ㅜ
ㅠ
ㅡ
ㅣ
가
각
간
갈
감
갑
값
갓
갔
강
갖
같
개
객
갠
갱
걍
거
건
걸
검
겁
것
겉
게
겐
겟
겠
겨
격
견
결
겹
겼
경
계
고
곡
곤
골
곱
곳
공
과
관
광
괜
괴
굉
교
구
국
군
굳
굴
굿
궁
권
귀
규
균
그
극
근
글
금
급
기
긴
길
김
깊
까
깎
깐
깔
깜
깝
깨
꺼
껄
껏
께
껴
꼈
꼬
꼭
꼴
꼽
꽃
꽝
꽤
꾸
꾼
꿀
꿈
꿔
뀌
끄
끈
끊
끌
끔
끝
끼
낀
낄
낌
나
낙
낚
난
날
남
납
낫
났
낭
낮
낳
내
낸
낼
냄
냈
냉
냐
냥
너
넌
널
넘
넣
네
넷
녀
년
념
녕
노
녹
논
놀
놈
농
높
놓
놔
놨
뇌
누
눈
뉴
느
는
늘
늙
능
늦
니
닉
닌
닐
님
닙
닝
다
닥
단
닫
달
닮
담
답
당
닿
대
댄
댓
더
덕
던
덜
덤
덩
데
덴
뎁
도
독
돈
돋
돌
동
돼
됐
되
된
될
됨
됩
됬
두
둘
둥
뒤
뒷
드
득
든
듣
들
듬
듭
듯
등
디
딘
딧
딩
따
딱
딴
딸
땅
때
땐
땜
땡
떄
떠
떡
떤
떨
떻
떼
또
똑
똥
뚝
뚱
뛰
뜨
뜩
뜬
뜻
라
락
란
랄
람
랍
랐
랑
래
랙
랜
램
랫
랬
략
량
러
럭
런
럴
럼
럽
렁
렇
레
렉
렌
려
력
련
렬
렵
렷
렸
령
례
로
록
론
롭
롯
롱
뢰
료
룡
루
룬
류
륜
률
륭
르
른
를
름
리
릭
린
릴
림
립
릿
링
마
막
만
많
말
맘
맙
맛
망
맞
맡
매
맥
맨
맹
머
먹
먼
멀
멈
멋
멍
메
멘
멜
며
면
멸
명
몇
모
목
몬
몰
몸
못
몽
묘
무
묵
문
묻
물
뭉
뭐
뭔
뭘
뮤
므
미
믹
민
믿
밀
밋
밌
밑
바
박
밖
반
받
발
밝
밤
밥
방
배
백
뱀
버
번
벌
범

## 2-4 토큰화하기

**GPT, BERT 입력값 만들기**

1단계 코랩 노트북 초기화하기

In [9]:
# 의존성 패키지 설치
!pip install ratsnlp

# 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


2단계 GPT 입력값 만들기

1. 토크나이저 준비
2. 이전 실습에서 만든 바이트 기준 BPE 어휘 집합(vocab.json)과 바이그램 쌍의 병합 우선순위(merge.txt)가 있어야 함

In [11]:
# GPT 토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/content/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

In [12]:
# GPT 토크나이저로 토큰화 하기

sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [14]:
# 토큰화 결과 확인. GPT 모델을 바이트 기준 BPE를 적용

tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [16]:
# GPT 모델 입력 만들기

batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",              # 문장의 최대 길이에 맞춰 패딩
    max_length=12,                     # 문장의 토큰 기준 최대 길이
    truncation=True,                   # 문장 잘림 허용 옵션
)

In [17]:
# batch_inputs 내용 확인

batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [18]:
# input_ids는 토큰화 결과를 가지고 각 토큰들을 인덱스로 바꾼 것
batch_inputs['input_ids']

# 어휘집합을 확인해보면 각 어휘 순서대로 나열. 이 순서가 바로 인덱스.
# 각 토큰을 인덱스로 변환하는 과정을 인덱싱이라고 함

# 모든 문장의 길이(토큰 수)가 12열로 맞춰진 것을 볼 수 있는데 이는 max_length 인자에 12를 넣었기 때문
# 이보다 짧은 문장은 인덱스 0을 붙임
# [PAD] 토큰은 일종의 더미 토큰으로 길이를 맞춰주는 역할을 함
# 두번째 행 문장2는 우너래 토큰 길이가 15였는데 12로 준 것. 이는 truncation=True 옵션 때문

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [19]:
# 일반 토큰이 자리하는 곳은 1로 패딩 토큰이 자리한 곳은 0으로 구분해 알려주는 장치 attention_mask

batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

3단계 BERT 입력값 만들기

In [21]:
#  BERT 토크나이저 선언

# BERT 모델의 입력값을 만드는 것.
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case=False,
)

In [22]:
# 예시 문장 3개를 워드피스 토크나이저로 토큰화
# BERT 토크나이저로 토큰화하기

sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [24]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [29]:
# BERT 모델 입력 만들기

batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [30]:
batch_inputs["input_ids"]

# 모든 문장 앞에 2, 끝에 3 붙었는데 이는 각각 CLS, SEP 토큰에 대응하는 인덱스

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1256, 16, 3],
 [2, 3274, 9507, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [31]:
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]